In [ ]:
import pandas as pd
import numpy as np

# for regression problems
from sklearn.linear_model import LinearRegression, Ridge

# for classification
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# to split and standarize the datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# to evaluate regression models
from sklearn.metrics import mean_squared_error

# to evaluate classification models
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# load the Titanic Dataset with a few variables for demonstration

data = pd.read_csv('titanic.csv', usecols = ['Age', 'Fare','Survived'])
data.head()

,Survived,Age,Fare
0,0,22.0,7.2500
1,1,38.0,71.2833
2,1,26.0,7.9250
3,1,35.0,53.1000
4,0,35.0,8.0500


In [ ]:
# let's look at the percentage of NA
data.isnull().mean()

Survived    0.000000
Age         0.198653
Fare        0.000000
dtype: float64

In [ ]:
# let's separate into training and testing set

X_train, X_test, y_train, y_test = train_test_split(data, data.Survived, test_size=0.3,
                                                    random_state=0)
X_train.shape, X_test.shape

((623, 3), (268, 3))

In [ ]:
# create variable indicating missingness

X_train['Age_NA'] = np.where(X_train['Age'].isnull(), 1, 0)
X_test['Age_NA'] = np.where(X_test['Age'].isnull(), 1, 0)

X_train.head()

,Survived,Age,Fare,Age_NA
857,1,51.0,26.5500,0
52,1,49.0,76.7292,0
386,0,1.0,46.9000,0
124,0,54.0,77.2875,0
578,0,NaN,14.4583,1


In [ ]:
# we can see that mean and median are similar. So I will replace with the median
X_train.Age.mean(), X_train.Age.median(),

(29.915338645418327, 29.0)

In [ ]:
# let's replace the NA with the median value in the training set
X_train['Age'].fillna(X_train.Age.median(), inplace=True)
X_test['Age'].fillna(X_train.Age.median(), inplace=True)

X_train.head(20)

,Survived,Age,Fare,Age_NA
857,1,51.0,26.5500,0
52,1,49.0,76.7292,0
386,0,1.0,46.9000,0
124,0,54.0,77.2875,0
578,0,29.0,14.4583,1
549,1,8.0,36.7500,0
118,0,24.0,247.5208,0
12,0,20.0,8.0500,0
157,0,30.0,8.0500,0
127,1,24.0,7.1417,0


### Logistic Regression

In [ ]:
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train))
X_test = pd.DataFrame(scaler.transform(X_test))

X_train.columns = ['Survived','Age','Fare','Age_NA']
X_test.columns = ['Survived','Age','Fare','Age_NA']

In [ ]:
# we compare the models built using Age filled with median, vs Age filled with median + additional
# variable indicating missingness

logit = LogisticRegression(random_state=44, C=1000) # c big to avoid regularization
logit.fit(X_train[['Age','Fare']], y_train)
print('Train set')
pred = logit.predict_proba(X_train[['Age','Fare']])
print('Logistic Regression roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
print('Test set')
pred = logit.predict_proba(X_test[['Age','Fare']])
print('Logistic Regression roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

logit = LogisticRegression(random_state=44, C=1000) # c big to avoid regularization
logit.fit(X_train[['Age','Age_NA', 'Fare']], y_train)
print('Train set')
pred = logit.predict_proba(X_train[['Age','Age_NA', 'Fare']])
print('Logistic Regression roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
print('Test set')
pred = logit.predict_proba(X_test[['Age','Age_NA', 'Fare']])
print('Logistic Regression roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

Train set
Logistic Regression roc-auc: 0.6794863451985858
Test set
Logistic Regression roc-auc: 0.7244940476190476
Train set
Logistic Regression roc-auc: 0.6762705798138868
Test set
Logistic Regression roc-auc: 0.7167857142857142


### Support Vector Machine

In [ ]:
# we compare the models built using Age filled with median, vs Age filled with median + additional
# variable indicating missingness

SVM_model = SVC(random_state=44, probability=True, max_iter=-1, kernel='linear')
SVM_model.fit(X_train[['Age', 'Fare']], y_train)
print('Train set')
pred = SVM_model.predict_proba(X_train[['Age', 'Fare']])
print('SVC roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
print('Test set')
pred = SVM_model.predict_proba(X_test[['Age', 'Fare']])
print('SCV roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

# model build using natural distributions

SVM_model = SVC(random_state=44, probability=True, max_iter=-1, kernel='linear')
SVM_model.fit(X_train[['Age','Age_NA', 'Fare']], y_train)
print('Train set')
pred = SVM_model.predict_proba(X_train[['Age','Age_NA', 'Fare']])
print('SVC roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
print('Test set')
pred = SVM_model.predict_proba(X_test[['Age','Age_NA', 'Fare']])
print('SVC roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

Train set
SVC roc-auc: 0.692978460337086
Test set
SCV roc-auc: 0.7418154761904762
Train set
SVC roc-auc: 0.6917203531376761
Test set
SVC roc-auc: 0.7360714285714286


In the titanic dataset, including a variable to indicate missingness for Age did not show an improvement in the performance of the logistic regression and barely the support vector machine.

### House Sale Dataset

In [ ]:
# we are going to train a model on the following variables,

cols_to_use = ['OverallQual', 'TotalBsmtSF', '1stFlrSF', 'GrLivArea','WoodDeckSF', 'BsmtUnfSF',
               'LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [ ]:
# let's load the House Sale Price dataset

data = pd.read_csv('houseprice.csv', usecols=cols_to_use+['SalePrice'])
print(data.shape)
data.head()

(1460, 10)


,LotFrontage,OverallQual,MasVnrArea,BsmtUnfSF,TotalBsmtSF,1stFlrSF,GrLivArea,GarageYrBlt,WoodDeckSF,SalePrice
0,65.0,7,196.0,150,856,856,1710,2003.0,0,208500
1,80.0,6,0.0,284,1262,1262,1262,1976.0,298,181500
2,68.0,7,162.0,434,920,920,1786,2001.0,0,223500
3,60.0,7,0.0,540,756,961,1717,1998.0,0,140000
4,84.0,8,350.0,490,1145,1145,2198,2000.0,192,250000


In [ ]:
# let's inspect the columns  with missing values
data.isnull().mean()

LotFrontage    0.177397
OverallQual    0.000000
MasVnrArea     0.005479
BsmtUnfSF      0.000000
TotalBsmtSF    0.000000
1stFlrSF       0.000000
GrLivArea      0.000000
GarageYrBlt    0.055479
WoodDeckSF     0.000000
SalePrice      0.000000
dtype: float64

In [ ]:
# let's separate into training and testing set

X_train, X_test, y_train, y_test = train_test_split(data, data.SalePrice, test_size=0.3,
                                                    random_state=0)
X_train.shape, X_test.shape

((1022, 10), (438, 10))

We observed that the numerical variables are not normally distributed. In particular, most of them apart from YearBuilt are skewed.

In [ ]:
# let's make a function to replace the NA with median or 0s

def impute_na(df, variable, median):
    df[variable+'_NA'] = np.where(df[variable].isnull(), 1, 0)
    df[variable].fillna(median, inplace=True)


In [ ]:
# let's look at the median of the variables with NA

X_train[['LotFrontage', 'MasVnrArea', 'GarageYrBlt']].median()

LotFrontage      69.0
MasVnrArea        0.0
GarageYrBlt    1979.0
dtype: float64

In [ ]:
# let's impute the NA with  the median
# remember that we need to impute with the median for the train set, and then propagate to test set

median = X_train.LotFrontage.median()
impute_na(X_train, 'LotFrontage', median)
impute_na(X_test, 'LotFrontage', median)

In [ ]:
median = X_train.MasVnrArea.median()
impute_na(X_train, 'MasVnrArea', median)
impute_na(X_test, 'MasVnrArea', median)

In [ ]:
median = X_train.GarageYrBlt.median()
impute_na(X_train, 'GarageYrBlt', median)
impute_na(X_test, 'GarageYrBlt', median)

In [ ]:
X_train.isnull().mean()

LotFrontage       0.0
OverallQual       0.0
MasVnrArea        0.0
BsmtUnfSF         0.0
TotalBsmtSF       0.0
1stFlrSF          0.0
GrLivArea         0.0
GarageYrBlt       0.0
WoodDeckSF        0.0
SalePrice         0.0
LotFrontage_NA    0.0
MasVnrArea_NA     0.0
GarageYrBlt_NA    0.0
dtype: float64

In [ ]:
# create a list with the untransformed columns
cols_to_use_no_na = X_train.columns[:-4]
cols_to_use_no_na

Index(['LotFrontage', 'OverallQual', 'MasVnrArea', 'BsmtUnfSF', 'TotalBsmtSF',
       '1stFlrSF', 'GrLivArea', 'GarageYrBlt', 'WoodDeckSF'],
      dtype='object')

In [ ]:
cols_to_use = list(X_train.columns)
cols_to_use.remove('SalePrice')
cols_to_use

['LotFrontage',
 'OverallQual',
 'MasVnrArea',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 'GrLivArea',
 'GarageYrBlt',
 'WoodDeckSF',
 'LotFrontage_NA',
 'MasVnrArea_NA',
 'GarageYrBlt_NA']

In [ ]:
# let's standarise the dataset
scaler = StandardScaler()
X_train_no_na = scaler.fit_transform(X_train[cols_to_use_no_na])
X_test_no_na = scaler.transform(X_test[cols_to_use_no_na])

scaler = StandardScaler()
X_train_all = scaler.fit_transform(X_train[cols_to_use])
X_test_all = scaler.transform(X_test[cols_to_use])

### Linear Regression

In [ ]:
# we compare the models built using Age filled with median, vs Age filled with median + additional
# variable indicating missingness

linreg = LinearRegression()
linreg.fit(X_train_no_na, y_train)
print('Train set')
pred = linreg.predict(X_train_no_na)
print('Linear Regression mse: {}'.format(mean_squared_error(y_train, pred)))
print('Test set')
pred = linreg.predict(X_test_no_na)
print('Linear Regression mse: {}'.format(mean_squared_error(y_test, pred)))
print()
linreg = LinearRegression()
linreg.fit(X_train_all, y_train)
print('Train set')
pred = linreg.predict(X_train_all)
print('Linear Regression mse: {}'.format(mean_squared_error(y_train, pred)))
print('Test set')
pred = linreg.predict(X_test_all)
print('Linear Regression mse: {}'.format(mean_squared_error(y_test, pred)))
print()

Train set
Linear Regression mse: 1161895545.483203
Test set
Linear Regression mse: 2212393764.7463093

Train set
Linear Regression mse: 1157194541.9444427
Test set
Linear Regression mse: 2197999822.6527514



In [ ]:
#  what is the difference in price estimated by the 2 models?

2212393764-2197999822

14393942

Here, when we build a model using the additional variable to capture missingness of data, we observe in the test set that the mse is smaller. This means that the difference between the real value and the estimated value is smaller, and thus our model performs better.

There is a difference of ~14 million between the model that replaces with the median and the one that uses median imputation in combination with the additional variables to capture missingness. So even when the difference in mse seems small, when we boil it down to business value, the impact is massive.

For a discussion on why the median imputation is not enough in this dataset, refer to lecture "Replacing NA by mean or median"

In [ ]:
# we compare the models built using Age filled with median, vs Age filled with median + additional
# variable indicating missingness

#  Ridge, is a regularised linear regression.

linreg = Ridge(random_state=30, max_iter=5, tol=100, alpha=10)
linreg.fit(X_train_no_na, y_train)
print('Train set')
pred = linreg.predict(X_train_no_na)
print('Ridge Regression mse: {}'.format(mean_squared_error(y_train, pred)))
print('Test set')
pred = linreg.predict(X_test_no_na)
print('Ridge Regression mse: {}'.format(mean_squared_error(y_test, pred)))
print()

linreg = Ridge(random_state=30, max_iter=5, tol=100, alpha=10)
linreg.fit(X_train_all, y_train)
print('Train set')
pred = linreg.predict(X_train_all)
print('Ridge Regression mse: {}'.format(mean_squared_error(y_train, pred)))
print('Test set')
pred = linreg.predict(X_test_all)
print('Ridge Regression mse: {}'.format(mean_squared_error(y_test, pred)))
print()

Train set
Ridge Regression mse: 1162112961.4104362
Test set
Ridge Regression mse: 2203311969.187996

Train set
Ridge Regression mse: 1157413427.915226
Test set
Ridge Regression mse: 2188469437.2202415



We observe the same conclusion when using regularised linear regression.